In [ ]:
# Adapted from https://medium.com/naukri-engineering/text-classification-using-bert-sklearn-and-pytorch-7665433b56c7

In [4]:
import numpy as np
import pandas as pd
import torch
import transformers as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel

In [2]:
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [6]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [28]:
max_len = 512

def tokenize_one_text(text):
    tokenized = tokenizer.encode(text, max_length=512, add_special_tokens=True)
    padded = np.array([tokenized + [0]*(max_len-len(tokenized))])
    attention_mask = np.where(padded != 0, 1, 0)

    input_ids = torch.tensor(padded).to(torch.int64)
    attention_mask = torch.tensor(attention_mask).to(torch.int64)

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    features = last_hidden_states[0][:,0,:].numpy()
    
    return features

In [ ]:
texts = df['Input.full_text'].to_list()
tokenized_arr = []
for text in texts:
    tokenized_text = tokenize_one_text(text)
    tokenized_arr.append(tokenized_text)

In [ ]:
labels = df['Input.deception_quadrant'].to_list()
train_features, test_features, train_labels, test_labels = train_test_split(tokenized_arr, labels) 
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)